# Test code

This notebook converts sg015 data from the Labrador Sea in September 2024 into OG1 format.

- OG1 format is a newly agreed format (since June 2024) for glider data sets from various platforms (e.g., Seaglider, Slocum, Seaexplorer).  It lives on github here: (https://github.com/OceanGlidersCommunity/OG-format-user-manual).
- OG1 manual: https://oceangliderscommunity.github.io/OG-format-user-manual/OG_Format.html


In [ ]:
import sys
import importlib
import yaml
sys.path.append('/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1')
sys.path.append('/Users/eddifying/Cloudfree/gitlab-cloudfree/glidertest')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from seagliderOG1 import fetchers
from seagliderOG1 import tools
from seagliderOG1 import plotters
from seagliderOG1 import readers
from seagliderOG1 import convertOG1
from seagliderOG1 import vocabularies
from seagliderOG1 import  attr_input
import glidertest as gt
import xarray as xr
import os
import warnings
warnings.simplefilter("ignore", category=Warning)


## Load Seaglider data in native format

1. Load multiple basestation files into a list of xarray datasets (`list_datasets`)
2. For (each) of these basestation datasets, split the dataset by dimension (`split_ds`)
3. Transform into OG1 format: dataset with dims `sg_data_point`
    - Change the dimension to `N_MEASUREMENTS`
    - Rename variables according to `vocabularies.standard_names` 
    - Assign attributes to variables (note that this *could* go wrong).  May need a function to handle each one, ideally based on a yaml file for the mission??
4. Add missing mandatory variables: 
    - from `gps_info`, the `LATITUDE_GPS`, `LONGITUDE_GPS` and `TIME_GPS`
    - Create `PROFILE_NUMBER` and `PHASE`
5. Update attributes
6. Retain the variables starting with `sg_cal` and check whether they vary over the mission (shouldn't)
6. Add sensors, using information in the `split_ds` with no dimensions
    - Need (from sg_cal_constants: `sg_cal` plus `volmax`, `vbd_cnts_per_cc`, `therm_expan`, `t_*`, `mass`, `hd_*`, `ctcor`, `cpcor`, `c_*`, `abs_compress`, `a`, `Tcor`, `Soc`, `Pcor`, `Foffset`)
    - Maybe also `reviewed`, `magnetic_variation` (which will change with position), `log_D_FLARE`, `flight_avg_speed_north` and `flight_avg_speed_east` also with `_gsm`, `depth_avg_curr_north` and `depth_avg_curr_east` also with `_gsm`, 
    `wlbb2f` - means sensor
    `sg_cal_mission_title`
    `sg_cal_id_str`
    `calibcomm_oxygen`
    `calibcomm`
    `sbe41` means ??
    `hdm_qc`
    `glider`


In [ ]:
importlib.reload(readers)
# Specify the server where data are located
if 0:
    input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/"
else:
    input_loc = '/Users/eddifying/Dropbox/data/sg015-ncei-download/'

# Load and concatenate all datasets from the server, optionally specifying the range of profiles to load
start_dive = 1
end_dive = 10

list_datasets = readers.read_basestation(input_loc, 1, 10)


In [ ]:
importlib.reload(convertOG1)
importlib.reload(vocabularies)
importlib.reload(plotters)

# Pick one basestation xarray dataset to work with
ds1 = list_datasets[0]

# Convert the dataset and output also variables not included
ds_new, attr_warnings, sg_cal, dc_other, dc_log = convertOG1.process_dataset(ds1)

# Add these to existing attributes
contrib_to_append = {
    'contributor_name': 'Eleanor Frajka-Williams',
    'contributor_email': 'eleanorfrajka@gmail.com',
    'contributor_role': 'Data scientist',
    'contributor_role_vocabulary': 'http://vocab.nerc.ac.uk/search_nvs/W08',
    'contributing_institutions': 'University of Hamburg - Institute of Oceanography',
    'contributing_institutions_vocabulary': 'https://edmo.seadatanet.org/report/1156',
    'contributing_institutions_role': 'Data scientist',
    'contributing_institutions_role_vocabulary': 'http://vocab.nerc.ac.uk/search_nvs/W08',
}

# Create the attributes in order
ordered_attributes = convertOG1.update_dataset_attributes(ds1, contrib_to_append)

for key, value in ordered_attributes.items():
    ds_new.attrs[key] = value

plotters.show_variables(ds_new)

## Save the final dataset

At the moment, since the plotters.show_attributes() and plotters.show_variables() are designed to work with netcdf files, I write the xarray dataset to netcdf before loading it and checking the attributes and variable names.

In [ ]:
plotters.show_attributes(ds_new)
## Save the dataset to a NetCDF file
# Check if the file exists and delete it if it does
output_file = os.path.join('../data', 'sg015_'+ds_new.start_date+'_delayed_temp.nc')
if os.path.exists(output_file):
    os.remove(output_file)

# Save the dataset to a NetCDF file
convertOG1.save_dataset(ds_new, output_file)


In [ ]:
importlib.reload(plotters)
plotters.show_variables(ds_new)